In [6]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')
project_id = 'my-project'
client = bigquery.Client(project = project_id)

Authenticated


Exploratory Data Analysis is the first step in exploring new or unknown dataset. It aims to describe the metadata, provide statistics and find outliers. Python Pandas package providea an easy way to perform EDA. However, not all data users are comfortable with Python. Dynamic SQL provides alternative workaround for EDA exploration.

[Kaggle dataset](https://www.kaggle.com/datasets/adammaus/predicting-churn-for-bank-customers) is used for the EDA. Data is preloaded into BQ table temp.churn_model

Step 1: Explore metadata about the dataset: get number of columns and records. (use in place of df.shape)

In [9]:
%%bigquery churn_table_medatadata --project my-project
SELECT
    count(distinct column_name) as count_columns
,  (select  count(*) from  `my-project.temp.churn_model`) as count_rows
FROM `my-project.temp.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'churn_model'
;


Query is running:   0%|          |

Downloading:   0%|          |

In [10]:
print (churn_table_medatadata)

   count_columns  count_rows
0             14       10000


Step 2: Explore metadata about the dataset: get columns and variable types. (use in place of df.types and df.columns)

In [13]:
%%bigquery churn_table_medatadata --project my-project
SELECT * EXCEPT(table_catalog, table_schema, is_generated, generation_expression, is_stored, is_updatable, clustering_ordinal_position, collation_name, column_default, rounding_mode)
FROM `my-project.temp.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'churn_model'
;

Query is running:   0%|          |

Downloading:   0%|          |

In [14]:
print (churn_table_medatadata)

     table_name      column_name  ordinal_position is_nullable data_type  \
0   churn_model        RowNumber                 1         YES     INT64   
1   churn_model       CustomerId                 2         YES     INT64   
2   churn_model          Surname                 3         YES    STRING   
3   churn_model      CreditScore                 4         YES     INT64   
4   churn_model        Geography                 5         YES    STRING   
5   churn_model           Gender                 6         YES    STRING   
6   churn_model              Age                 7         YES     INT64   
7   churn_model           Tenure                 8         YES     INT64   
8   churn_model          Balance                 9         YES   FLOAT64   
9   churn_model    NumOfProducts                10         YES     INT64   
10  churn_model        HasCrCard                11         YES     INT64   
11  churn_model   IsActiveMember                12         YES     INT64   
12  churn_mo

Step 3: Count unique values for each column. (use in place of df.nunique)

In [15]:
%%bigquery churn_table_medatadata --project my-project
DECLARE columns ARRAY<STRING>;
DECLARE query STRING;
SET columns = (
  WITH all_columns AS (
    SELECT column_name
    FROM `my-project.temp.INFORMATION_SCHEMA.COLUMNS`
    WHERE table_name = 'churn_model'
  )
  SELECT ARRAY_AGG((column_name) ) AS columns
  FROM all_columns
)
;

SET query = (select STRING_AGG('(select count(distinct '||x||')  from `my-project.temp.churn_model`) '||x ) AS string_agg from unnest(columns) x );
EXECUTE IMMEDIATE
"SELECT  "|| query
;

Query is running:   0%|          |

Downloading:   0%|          |

In [16]:
print (churn_table_medatadata)

   RowNumber  CustomerId  Surname  CreditScore  Geography  Gender  Age  \
0      10000       10000     2932          460          3       2   70   

   Tenure  Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0      11     6382              4          2               2             9999   

   Exited  
0       2  


Step 4: Describe dataset: return statistical information about the dataset. (use in place of df.describe()). These calculations can only be used on numeric fields.  

In [17]:
%%bigquery churn_table_medatadata --project my-project
DECLARE columns ARRAY<STRING>;
DECLARE query1, query2, query3, query4, query5, query6, query7 STRING;
SET columns = (
  WITH all_columns AS (
    SELECT column_name
    FROM `my-project.temp.INFORMATION_SCHEMA.COLUMNS`
    WHERE table_name = 'churn_model'
        and  data_type IN ('INT64','FLOAT64')
  )
  SELECT ARRAY_AGG((column_name) ) AS columns
  FROM all_columns
);

SET query1 = (select STRING_AGG('(select stddev( '||x||')  from `my-project.temp.churn_model`) '||x ) AS string_agg from unnest(columns) x );
SET query2 = (select STRING_AGG('(select avg( '||x||')  from `my-project.temp.churn_model`) '||x ) AS string_agg from unnest(columns) x );
SET query3 = (select STRING_AGG('(select PERCENTILE_CONT( '||x||', 0.5) over()  from `my-project.temp.churn_model` limit 1) '||x ) AS string_agg from unnest(columns) x );
SET query4 = (select STRING_AGG('(select PERCENTILE_CONT( '||x||', 0.25) over()  from `my-project.temp.churn_model` limit 1) '||x ) AS string_agg from unnest(columns) x );
SET query5 = (select STRING_AGG('(select PERCENTILE_CONT( '||x||', 0.75) over()  from `my-project.temp.churn_model` limit 1) '||x ) AS string_agg from unnest(columns) x );
SET query6 = (select STRING_AGG('(select max( '||x||')  from `my-project.temp.churn_model`) '||x ) AS string_agg from unnest(columns) x );
SET query7 = (select STRING_AGG('(select min( '||x||')  from `my-project.temp.churn_model`) '||x ) AS string_agg from unnest(columns) x );

EXECUTE IMMEDIATE (
"SELECT 'stddev' ,"|| query1 || " UNION ALL " ||
"SELECT 'mean'   ,"|| query2 || " UNION ALL " ||
"SELECT 'median' ,"|| query3 || " UNION ALL " ||
"SELECT '0.25' ,"|| query4 || " UNION ALL " ||
"SELECT '0.75' ,"|| query5 || " UNION ALL " ||
"SELECT 'max' ,"|| query6 || " UNION ALL " ||
"SELECT 'min' ,"|| query7
)
;

Query is running:   0%|          |

Downloading:   0%|          |

In [18]:
print (churn_table_medatadata)

      f0_    RowNumber    CustomerId  CreditScore        Age     Tenure  \
0  stddev   2886.89568  7.193619e+04    96.653299  10.487806   2.892174   
1    0.75   7500.25000  1.575323e+07   718.000000  44.000000   7.000000   
2    0.25   2500.75000  1.562853e+07   584.000000  32.000000   3.000000   
3     min      1.00000  1.556570e+07   350.000000  18.000000   0.000000   
4     max  10000.00000  1.581569e+07   850.000000  92.000000  10.000000   
5  median   5000.50000  1.569074e+07   652.000000  37.000000   5.000000   
6    mean   5000.50000  1.569094e+07   650.528800  38.921800   5.012800   

         Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0   62397.405202       0.581654    0.45584        0.499797     57510.492818   
1  127644.240000       2.000000    1.00000        1.000000    149388.247500   
2       0.000000       1.000000    0.00000        0.000000     51002.110000   
3       0.000000       1.000000    0.00000        0.000000        11.580000   
4  2

Step 5: Check outliers: check if there are outlines for field "Tenure" that are more than 2 standard deviations from the mean.

In [19]:
%%bigquery churn_table_medatadata --project my-project
DECLARE lower, upper, mean FLOAT64;
SET mean = (select avg( Tenure)  from `my-project.temp.churn_model`);
SET lower = mean - 2 * (select stddev( Tenure)  from `my-project.temp.churn_model`);
SET upper = mean + 2 * (select stddev( Tenure)  from `my-project.temp.churn_model`);
EXECUTE IMMEDIATE (
"SELECT * from `my-project.temp.churn_model` WHERE Tenure >"|| upper ||" OR Tenure < " || lower
)
;

Query is running:   0%|          |

Downloading: |          |

No outliers were returned.

Step 6: Explore relationship between variables - build correlation matrix. (use in place of df.corr() and heatmap grid)

In [20]:
%%bigquery churn_table_medatadata --project my-project
DECLARE columns ARRAY<STRING>;
DECLARE query STRING DEFAULT '';
DECLARE i,j INT64 DEFAULT 0;

SET columns = (
  WITH all_columns AS (
    SELECT column_name
    FROM `my-project.temp.INFORMATION_SCHEMA.COLUMNS`
    WHERE table_name = 'churn_model'
        and column_name not in ('RowNumber', 'CustomerId', 'Surname')
        and  data_type IN ('INT64','FLOAT64')
  )
  SELECT ARRAY_AGG((column_name) ) AS columns
  FROM all_columns
);

LOOP
    SET i = i + 1;

    IF i > ARRAY_LENGTH(columns) THEN
        LEAVE;
    END IF;

    IF i > 1 THEN
        SET query = query || ' UNION ALL ';
    END IF;
        SET query = query || ' SELECT '|| i || ","||"'"||columns[ORDINAL(i)]|| "'" ;

        LOOP
            SET j = j + 1;

            IF j > ARRAY_LENGTH(columns) THEN
                LEAVE;
            END IF;

            SET query = query || ' , round(corr('|| columns[ORDINAL(i)] ||','|| columns[ORDINAL(j)] ||'),4)  as ' ||" "||columns[ORDINAL(j)]|| " "  ;
        END LOOP;
        SET query = query || ' FROM  `my-project.temp.churn_model` ' ;
        SET j = 0;


END LOOP;

Query is running:   0%|          |

In [22]:
%%bigquery churn_table_medatadata --project my-project
SELECT 1,'CreditScore' , round(corr(CreditScore,CreditScore),4)  as  CreditScore  , round(corr(CreditScore,Age),4)  as  Age  , round(corr(CreditScore,Tenure),4)  as  Tenure  , round(corr(CreditScore,Balance),4)  as  Balance  , round(corr(CreditScore,NumOfProducts),4)  as  NumOfProducts  , round(corr(CreditScore,HasCrCard),4)  as  HasCrCard  , round(corr(CreditScore,IsActiveMember),4)  as  IsActiveMember  , round(corr(CreditScore,EstimatedSalary),4)  as  EstimatedSalary  , round(corr(CreditScore,Exited),4)  as  Exited  FROM  `my-project.temp.churn_model`
UNION ALL
SELECT 2,'Age' , round(corr(Age,CreditScore),4)  as  CreditScore  , round(corr(Age,Age),4)  as  Age  , round(corr(Age,Tenure),4)  as  Tenure  , round(corr(Age,Balance),4)  as  Balance  , round(corr(Age,NumOfProducts),4)  as  NumOfProducts  , round(corr(Age,HasCrCard),4)  as  HasCrCard  , round(corr(Age,IsActiveMember),4)  as  IsActiveMember  , round(corr(Age,EstimatedSalary),4)  as  EstimatedSalary  , round(corr(Age,Exited),4)  as  Exited  FROM  `my-project.temp.churn_model`  UNION ALL  SELECT 3,'Tenure' , round(corr(Tenure,CreditScore),4)  as  CreditScore  , round(corr(Tenure,Age),4)  as  Age  , round(corr(Tenure,Tenure),4)  as  Tenure  , round(corr(Tenure,Balance),4)  as  Balance  , round(corr(Tenure,NumOfProducts),4)  as  NumOfProducts  , round(corr(Tenure,HasCrCard),4)  as  HasCrCard  , round(corr(Tenure,IsActiveMember),4)  as  IsActiveMember  , round(corr(Tenure,EstimatedSalary),4)  as  EstimatedSalary  , round(corr(Tenure,Exited),4)  as  Exited  FROM  `my-project.temp.churn_model`  UNION ALL  SELECT 4,'Balance' , round(corr(Balance,CreditScore),4)  as  CreditScore  , round(corr(Balance,Age),4)  as  Age  , round(corr(Balance,Tenure),4)  as  Tenure  , round(corr(Balance,Balance),4)  as  Balance  , round(corr(Balance,NumOfProducts),4)  as  NumOfProducts  , round(corr(Balance,HasCrCard),4)  as  HasCrCard  , round(corr(Balance,IsActiveMember),4)  as  IsActiveMember  , round(corr(Balance,EstimatedSalary),4)  as  EstimatedSalary  , round(corr(Balance,Exited),4)  as  Exited  FROM  `my-project.temp.churn_model`  UNION ALL  SELECT 5,'NumOfProducts' , round(corr(NumOfProducts,CreditScore),4)  as  CreditScore  , round(corr(NumOfProducts,Age),4)  as  Age  , round(corr(NumOfProducts,Tenure),4)  as  Tenure  , round(corr(NumOfProducts,Balance),4)  as  Balance  , round(corr(NumOfProducts,NumOfProducts),4)  as  NumOfProducts  , round(corr(NumOfProducts,HasCrCard),4)  as  HasCrCard  , round(corr(NumOfProducts,IsActiveMember),4)  as  IsActiveMember  , round(corr(NumOfProducts,EstimatedSalary),4)  as  EstimatedSalary  , round(corr(NumOfProducts,Exited),4)  as  Exited  FROM  `my-project.temp.churn_model`  UNION ALL  SELECT 6,'HasCrCard' , round(corr(HasCrCard,CreditScore),4)  as  CreditScore  , round(corr(HasCrCard,Age),4)  as  Age  , round(corr(HasCrCard,Tenure),4)  as  Tenure  , round(corr(HasCrCard,Balance),4)  as  Balance  , round(corr(HasCrCard,NumOfProducts),4)  as  NumOfProducts  , round(corr(HasCrCard,HasCrCard),4)  as  HasCrCard  , round(corr(HasCrCard,IsActiveMember),4)  as  IsActiveMember  , round(corr(HasCrCard,EstimatedSalary),4)  as  EstimatedSalary  , round(corr(HasCrCard,Exited),4)  as  Exited  FROM  `my-project.temp.churn_model`  UNION ALL  SELECT 7,'IsActiveMember' , round(corr(IsActiveMember,CreditScore),4)  as  CreditScore  , round(corr(IsActiveMember,Age),4)  as  Age  , round(corr(IsActiveMember,Tenure),4)  as  Tenure  , round(corr(IsActiveMember,Balance),4)  as  Balance  , round(corr(IsActiveMember,NumOfProducts),4)  as  NumOfProducts  , round(corr(IsActiveMember,HasCrCard),4)  as  HasCrCard  , round(corr(IsActiveMember,IsActiveMember),4)  as  IsActiveMember  , round(corr(IsActiveMember,EstimatedSalary),4)  as  EstimatedSalary  , round(corr(IsActiveMember,Exited),4)  as  Exited  FROM  `my-project.temp.churn_model`  UNION ALL  SELECT 8,'EstimatedSalary' , round(corr(EstimatedSalary,CreditScore),4)  as  CreditScore  , round(corr(EstimatedSalary,Age),4)  as  Age  , round(corr(EstimatedSalary,Tenure),4)  as  Tenure  , round(corr(EstimatedSalary,Balance),4)  as  Balance  , round(corr(EstimatedSalary,NumOfProducts),4)  as  NumOfProducts  , round(corr(EstimatedSalary,HasCrCard),4)  as  HasCrCard  , round(corr(EstimatedSalary,IsActiveMember),4)  as  IsActiveMember  , round(corr(EstimatedSalary,EstimatedSalary),4)  as  EstimatedSalary  , round(corr(EstimatedSalary,Exited),4)  as  Exited  FROM  `my-project.temp.churn_model`  UNION ALL  SELECT 9,'Exited' , round(corr(Exited,CreditScore),4)  as  CreditScore  , round(corr(Exited,Age),4)  as  Age  , round(corr(Exited,Tenure),4)  as  Tenure  , round(corr(Exited,Balance),4)  as  Balance  , round(corr(Exited,NumOfProducts),4)  as  NumOfProducts  , round(corr(Exited,HasCrCard),4)  as  HasCrCard  , round(corr(Exited,IsActiveMember),4)  as  IsActiveMember  , round(corr(Exited,EstimatedSalary),4)  as  EstimatedSalary  , round(corr(Exited,Exited),4)  as  Exited  FROM  `my-project.temp.churn_model`  order by 1


Query is running:   0%|          |

Downloading:   0%|          |

In [23]:
print (churn_table_medatadata)

   f0_              f1_  CreditScore     Age  Tenure  Balance  NumOfProducts  \
0    1      CreditScore       1.0000 -0.0040  0.0008   0.0063         0.0122   
1    2              Age      -0.0040  1.0000 -0.0100   0.0283        -0.0307   
2    3           Tenure       0.0008 -0.0100  1.0000  -0.0123         0.0134   
3    4          Balance       0.0063  0.0283 -0.0123   1.0000        -0.3042   
4    5    NumOfProducts       0.0122 -0.0307  0.0134  -0.3042         1.0000   
5    6        HasCrCard      -0.0055 -0.0117  0.0226  -0.0149         0.0032   
6    7   IsActiveMember       0.0257  0.0855 -0.0284  -0.0101         0.0096   
7    8  EstimatedSalary      -0.0014 -0.0072  0.0078   0.0128         0.0142   
8    9           Exited      -0.0271  0.2853 -0.0140   0.1185        -0.0478   

   HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0    -0.0055          0.0257          -0.0014 -0.0271  
1    -0.0117          0.0855          -0.0072  0.2853  
2     0.0226         -0.0284   